<h1>VacationPy: An exercise in making API calls to analyze data.</h1>

This code will use Geocoding and API calls to OpenWeather to analyze data from 500 Worldwide cities in order to identify an ideal vacation spot in terms of pleasant weather.

In [1]:
# Dependencies
import requests
import json
from citipy import citipy
import numpy as np
from config import owkey
import geoip2.database
import geoip2.webservice
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
import gmaps
from config import gkey
import pprint
import time
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [16]:
#Import CSV file with city data
file="worldcitiespop/worldcitiespop_reduced.csv"
cities_df=pd.read_csv(file)

In [17]:
#Extract a random sample (without replacement) of 500 cities from the CSV file
simpcities_df = cities_df[["Latitude","Longitude"]]
cities_dict=simpcities_df.set_index("Latitude")["Longitude"].to_dict()
random_500=dict(random.sample(cities_dict.items(), 500))

In [4]:
#Create a dataframe with weather info

weather_df= pd.DataFrame({
    "Name": [], 
    "Temperature (F)": [],
    "Max Temperature (F)":[],
    "Humidity (%)": [], 
    "Cloudiness (%)": [], 
    "Windspeed (mph)": [], 
    
})

#Loop through the dataframe to make API calls and get the weather in 500 cities

call_counter=0
for lat in random_500:
    api_call = "http://api.openweathermap.org/data/2.5/weather?lat=" + str(lat) + "&lon=" + str(random_500[lat]) + "&units=imperial" + "&appid=" + owkey
    api_response = requests.get(api_call)
    api_json = api_response.json()
    name=api_json["name"]
    temp=api_json["main"]["temp"]
    temp_max=api_json["main"]["temp_max"]
    humidity=api_json["main"]["humidity"]
    clouds=api_json["clouds"]["all"]
    speed=api_json["wind"]["speed"]
    
    #append the weather values to the dataframe
    weather_df=weather_df.append({'Name' : name , 
                                  'Temperature (F)' : temp, 
                                  "Humidity (%)": humidity, 
                                  "Cloudiness (%)": clouds, 
                                  "Windspeed (mph)": speed,
                                  "Max Temperature (F)":temp_max,
                                  "Latitude": lat, 
                                  "Longitude": random_500[lat] } , ignore_index=True)
    
    call_counter+=1
    if call_counter==19:
        time.sleep(1)
        call_counter=0
        
        continue

In [5]:
weather_df["Distance From Equator"]=abs(weather_df["Latitude"])
weather_df

,Name,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Windspeed (mph),Latitude,Longitude,Distance From Equator
0,Chadi,62.13,62.13,76.0,80.0,6.67,24.852532,116.593933,24.852532
1,Teufen AR,26.83,32.00,64.0,90.0,4.61,47.390723,9.389564,47.390723
2,Hallennes-lez-Haubourdin,44.40,45.00,87.0,90.0,6.91,50.626647,2.957199,50.626647
3,Beaune-la-Rolande,31.89,34.00,89.0,78.0,5.57,48.071245,2.431398,48.071245
4,Gaoya,40.26,40.26,18.0,100.0,2.91,35.003629,107.200853,35.003629
...,...,...,...,...,...,...,...,...,...
495,Kamenický Šenov,26.98,30.00,92.0,97.0,5.95,50.764701,14.437493,50.764701
496,Ålestrup,38.17,39.00,81.0,11.0,17.27,56.584350,9.572835,56.584350
497,Shuangnian,22.35,22.35,53.0,0.0,21.68,39.644943,112.064247,39.644943
498,Cóbreces,46.31,46.99,66.0,100.0,3.00,43.387450,-4.213608,43.387450


In [6]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
weather_df = weather_df.dropna()
humidity_weight = weather_df["Humidity (%)"].astype(float)

In [7]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig 

Figure(layout=FigureLayout(height='420px'))

In [8]:
#Filter the data to find the locatons with the best weather

#----Filter by temperature
newlocs_df=weather_df.loc[weather_df["Max Temperature (F)"]>70]
newlocs_df=newlocs_df.loc[newlocs_df["Max Temperature (F)"]<80]

#----Filter by windspeed
newlocs_df=newlocs_df.loc[newlocs_df["Windspeed (mph)"]<10]


#----Filter by cloudiness
newlocs_df=newlocs_df.loc[newlocs_df["Cloudiness (%)"]==0]


In [9]:
#Build new heat map of the (reduced) list of ideal weather locations

newlocs = newlocs_df[["Latitude", "Longitude"]].astype(float)
humidity_weight2 = newlocs_df["Humidity (%)"].astype(float)

fig2 = gmaps.figure()
heat_layer2 = gmaps.heatmap_layer(newlocs, weights=humidity_weight2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig2.add_layer(heat_layer2)

fig2

Figure(layout=FigureLayout(height='420px'))

In [11]:
#Use Geocode API to loop through the list of ideal weather locations and get
#info about the nearest hotel within 50,000 meters of each (the homework said 5,000, but this yielded few results).

##NOTE: because data is sampled randomly and Geocode data is incomplete, sometimes missing values are returned---

#----Create new columns in the dataframe to hold new info

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
newlocs_df["City"] = ""
newlocs_df["Country"]=""
newlocs_df["City"]=""
#newlocsindex=newlocs_df.setindex("Name")


#----Get the nearest hotel by latitude and longitude
for index, row in newlocs_df.iterrows():
    
    hotelzone_lat = row['Latitude']
    hotelzone_long=row['Longitude']
    
    params = {
    "location": str(hotelzone_lat)+","+str(hotelzone_long),
    "radius": 50000,
    "input": "",
    "type":"lodging",
    "key": gkey}


    
    response = requests.get(base_url, params=params).json()
    results = response['results']

#----Try to print the hotel name, also save that to the dataframe

    try:
        print(f"Hotel name is {results[0]['name']}.")
        
        newlocs_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Hotel name not found.")
        
#----Make an new API call to collect country and city     
    newbase_url="https://maps.googleapis.com/maps/api/geocode/json?"
    
    params = {
    "latlng": str(hotelzone_lat)+","+str(hotelzone_long),
    "key": gkey}
    

#----Try to print the hotel city and country, also save that to the dataframe     
    try:
        response2 = requests.get(newbase_url, params=params).json()
        city= newlocs_df.loc[index, 'City']=response2["results"][0]["address_components"][2]["long_name"]
        newlocs_df.loc[index, 'City'] = city
        print(f"City is {city}.")
    except:
        print("City not found.")
    try:
        country= newlocs_df.loc[index, 'Country']=response2["results"][0]["address_components"][5]["long_name"]
        newlocs_df.loc[index, 'Country'] = country
        print(f"Country is {country}.")
        
    except (KeyError, IndexError):
        print("Country not found.")
        
    print("------------")
    

Hotel name is Inter Plaza Hotel.
City is São Paulo.
Country not found.
------------
Hotel name is Hampton by Hilton Guarulhos Airport.
City is Santa Isabel.
Country is 07500-000.
------------
Hotel name is Kone Diakaridia De Manankoro Mali.
City is Odienné.
Country not found.
------------


In [14]:
#Create a column in the dataframe that combines the relevant info so that I can drop it in the infobox

newlocs_df["BoxStuff"]= (newlocs_df["Hotel Name"] + ": "+ newlocs_df["City"]) + ", " +newlocs_df["Country"]

In [15]:
#Make those infoboxes

boxes = newlocs_df["BoxStuff"].tolist()

# Create a map layer that flags the hotels and puts my infoboxes inside them

last_locs = newlocs_df[["Latitude", "Longitude"]].astype(float)

hotel_layer = gmaps.symbol_layer(
    last_locs, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[item for item in boxes])

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 27.417885, 'lng': 30.777718},
   'viewport': {'northeast': {'lat': 27.4192845802915,
     'lng': 30.7790669802915},
    'southwest': {'lat': 27.4165866197085, 'lng': 30.7763690197085}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
  'name': 'Mikhail Wadih',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 3712,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108042338542311890753">ميخائيل وديع</a>'],
    'photo_reference': 'ATtYBwIKTHI7myZ-YZnsERy5C3AbdQTkdKg-B9Lys7paT8jCVFR44FxTMCMMR1R4SakxSXhi8clRe8bIVwnwO2OfM7ie9GvG6G-gIihbjitWfIoTbrxkxT6qJHuAsRtg5UhNX7qsq5S4NISwNd6K1SmsvLiYRn-QXr2yDJ7Y6h9bH2dLwjRq',
    'width': 5568}],
  'place_id': 'ChIJr0J5pPXkRBQRBUuWlZQ2jb4',
  'plus_code': {'compound_code': 'CQ9H+53 Al Kossia, Egypt',
   'global_code': '7GVGCQ9H+53'},
  'rating': 4.3,
  'reference': 'ChIJr0J5pPXkRBQRBUuWlZQ2jb4',
  